# ipywidgets

> [`ipywidgets`](https://github.com/jupyter-widgets/ipywidgets) provide observable, interactive experiences for Jupyter kernels. 

In [ ]:
NAME = "ipywidgets"
GH_ORG = "jupyter-widgets"
JS_NS = "@jupyter-widgets"
PY_PROVIDES = [NAME, "jupyterlab_widgets", "widgetsnbextension"]
URL = f"https://github.com/{GH_ORG}/{NAME}"

In [ ]:
import importnb

with importnb.Notebook():
    from jupyak.tasks import __comm, __ipython, __jupyterlab, __lumino, __traitlets
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

## schema

In [ ]:
py = "python"
jlw = f"{py}/jupyterlab_widgets"
ipw = f"{py}/ipywidgets"
nbw = f"{py}/widgetsnbextension"
py_pkgs = [ipw, jlw, nbw]
jlw_lib_js = f"{jlw}/lib/index.js"
jlw_ext = f"{jlw}/labextension"
jlw_ext_pj = f"{jlw_ext}/{W.PACKAGE_JSON}"
css_pkgs = [
    "controls",
]
src_pkgs = [
    "base",
    "controls",
    "base-manager",
    "html-manager",
    "output",
]

In [ ]:
JS_DIST_TASKS = {}

all_tsbi = []
for i, pkg in enumerate(src_pkgs):
    pkg_tsbi = f"./packages/{pkg}/{W.TSCONFIG_BUILDINFO}"
    all_tsbi += [pkg_tsbi]

    task = {
        "name": "build",
        "actions": [["yarn", "build:src"]],
        "file_dep": [
            f"./packages/{pkg}/src/**/*.ts",
            *([] if not i else [f"./packages/{src_pkgs[i-1]}/{W.TSCONFIG_BUILDINFO}"]),
        ],
        "targets": [pkg_tsbi],
    }
    if pkg in css_pkgs:
        task["actions"] += [["yarn", "build:css"]]
        task["file_dep"] += [f"./packages/{pkg}/css/**/*.less"]
        task["targets"] += [
            {"glob_neighbor": [f"packages/{pkg}/css/*.less", "../{stem}.css"]}
        ]
    JS_DIST_TASKS[f"./packages/{pkg}"] = [task]

JS_DIST_TASKS[f"./{jlw}"] = [
    {
        "name": "build",
        "actions": [["yarn", "build:lib"]],
        "file_dep": all_tsbi,
        "targets": [f"{jlw}/{W.TSCONFIG_BUILDINFO}", jlw_lib_js],
    },
]

In [ ]:
LAB_EXT = {
    f"./{jlw}/": {
        "file_dep": [f"packages/*/{W.TSCONFIG_BUILDINFO}", jlw_lib_js],
        "targets": [jlw_ext_pj],
    },
}

In [ ]:
DEFAULTS = {
    "name": NAME,
    "github": {"url": URL},
    "js": {
        "dependencies": [
            __lumino.NAME,
            __jupyterlab.NAME,
        ],
        "tasks": JS_DIST_TASKS,
        "dist_exclude_patterns": ["/jupyterlab_widgets", "/example"],
    },
    "py": {
        "pyproject_tomls": [f"{py_pkg}/{W.PYPROJECT_TOML}" for py_pkg in py_pkgs],
        "dependencies": [
            __jupyterlab.NAME,
            __ipython.NAME,
            __comm.NAME,
            __traitlets.NAME,
        ],
        "lab_extensions": LAB_EXT,
    },
    "lite": {
        "wheel_file_dep": {"jupyterlab_widgets": [jlw_ext_pj]},
        "wheel": True,
        "skip_wheel_patterns": ["widgetsnbextension"],
    },
}

In [ ]:
@Y.Yak.repo(
    NAME,
    tuple(DEFAULTS["py"]["dependencies"] + DEFAULTS["js"]["dependencies"]),
)
def default_factory(yak: Y.Yak):
    return DEFAULTS